In [ ]:
'''
******EEI100
'''
import os
import sys
import cv2 as cv
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import torchvision.transforms as transforms
#CV2  4.1.2可以用绝对地址，新版本只能使用相对路径
data_dir = 'data/Power_2023'
dataset_train = "train"
dataset_val = "val"
dataset_test = "test"
cls_train = 60
cls_val = 20
cls_test = 20


In [ ]:
#测试用，下面直接写成函数read_lablel()
fname = dataset_train
path = os.path.join(data_dir,fname)
result = []
for dir_item in os.listdir(path):
    fall_path = os.path.abspath(os.path.join(path, dir_item))
    if os.path.isdir(fall_path):
        result.append(dir_item)
        
print(result)
print(len(result))

In [ ]:
def read_lablel(fname):
    result = []
    path = os.path.join(data_dir,fname)
    for dir_item in os.listdir(path):
        fall_path = os.path.abspath(os.path.join(path, dir_item))
        if os.path.isdir(fall_path):
            result.append(dir_item)
    return result

train_lablel=read_lablel(dataset_train)    
print(len(train_lablel))

In [ ]:
train_label = read_lablel(dataset_train)    
val_label = read_lablel(dataset_val)    
test_label = read_lablel(dataset_test)    
print(len(train_label))
print(len(val_label))
print(len(test_label))

In [ ]:
#求类别，并合并label
labels_list = train_label +  test_label 
print('数据集label总数：',len(labels_list))
print(labels_list)
print('train,val交集： ',set(train_label) & set(val_label))
print('train,test交集： ',set(train_label) & set(test_label))
#print('val,test交集： ',set(val_label) & set(test_label))

In [ ]:
#将类别转化为数字标号
labels_dict = {}
i = 1
for l in labels_list:
    if l not in labels_dict:
        labels_dict[l] = i
        i = i+1
print(labels_dict)

In [ ]:
#测试读取 图片文件
filepath = os.path.join(data_dir, dataset_train, train_label[0])
images = []
for image_item in os.listdir(filepath):
    images.append(image_item)
print(len(images))
print(images[0])
fall_path = os.path.join(filepath, images[0])
#image = cv.imread(fall_path)#读取图片
#print(image.shape)
image = Image.open(fall_path)
image.show()

In [ ]:
"""
读取图像文件，存入list中
下面直接封装成  get_image(path,_data)函数
"""
images = []
labels = []
i = 0
with tqdm(total=1942) as pbar:
    pbar.set_description('Processing:')
    for fname in train_lablel:
        filepath = os.path.join(data_dir, dataset_train, fname)
        for image_item in os.listdir(filepath):
            fall_path = os.path.join(filepath, image_item)
            image = cv.imread(fall_path)#读取图片
            images.append(image)
            labels.append(labels_dict[fname])
            pbar.update(1)#更新进度条
            i+=1
            if i==1:
                img0=Image.open(fall_path)
                img0.show()
                #print(fname+': '+str(labels_dict[fname]))

print(len(images))
print(set(train_lablel))

In [ ]:
print(images[0].shape,labels[0])
#cv.imshow("image label: %s" %labels[0],images[0])

#cv.waitKey(0)

In [ ]:
#对应图像小于crop高宽的,筛选出来
def filter(imgs,crop_size):
    return [img for img in imgs if (
        img.shape[0] < crop_size[0] or img.shape[1] < crop_size[1])]

In [ ]:
small_image = filter(images,[84,84])
print('小图片数量：',len(small_image))
if len(small_image) > 0:
    print(small_image[0].shape)
    Image.fromarray(small_image[0]).show()
    Image.fromarray(cv.resize(small_image[0],(84,84))).show()

In [ ]:
"""
读取图像文件，存入list中1942,891,891
"""
def get_image(label_data,cls_data,data_set):
    images = []
    labels = []
    crop_size = [84,84]
    with tqdm(total=891) as pbar:
        pbar.set_description('Processing:')
        for fname in label_data:
            filepath = os.path.join(data_dir, data_set, fname)
            for image_item in os.listdir(filepath):
                fall_path = os.path.join(filepath, image_item)
                image = cv.imread(fall_path)#读取图片
                image = cv.resize(image,(84,84))#resize成（84，84）
                images.append(image)
                labels.append(labels_dict[fname])
                pbar.update(1)#更新进度条
    return images, labels

In [ ]:
"""
写入.pickle文件
"""
#images, labels = get_image(train_label,cls_train,dataset_train)#替换上面读取的txt数据
#images, labels = get_image(val_label,cls_val,dataset_val)
images, labels = get_image(test_label,cls_test,dataset_test)
images = np.asarray(images,dtype=object)
print(images.shape)
small_image = filter(images,[84,84])
print('小图片数量：',len(small_image))
image_dic = {'data': images, 'labels' : labels}
#train  Power_2023_80_category_split_train_phase_train.pickle
#val  Power_2023_80_category_split_train_phase_val.pickle
#test  Power_2023_80_category_split_test.pickle
with open('Power_2023_80_category_split_test.pickle', 'wb') as f:
    pickle.dump(image_dic, f)
    f.close()

In [ ]:
"""
读取.pickle文件
"""
#path='Power_2023_80_category_split_train_phase_val.pickle'
#path='Power_2023_80_category_split_train_phase_train.pickle'
path='Power_2023_80_category_split_test.pickle'
with open(path,'rb') as f:
    data = pickle.load(f, encoding='bytes')
    imgs = data['data']
    labels = data['labels']
print(len(imgs))
print(imgs[0].shape)
print(type(labels[0]))

In [ ]:
#测试生成的数据正确不
def test(image_dataset,image_label):
    print(image_label[:10])
    plt.figure(figsize=(8,8))
    for i in range(10):
        plt.subplot(5,2,i+1)
        plt.imshow(np.asarray(image_dataset[i]).astype('uint8'))
        plt.show()
    
test(imgs,labels)

In [ ]:
item = 0
img = np.asarray(imgs[item]).astype('uint8')
print(len(img))
img = Image.fromarray(img)

img.show()

In [ ]:
slabels = np.asarray(labels)
print(slabels)
np.min(slabels)
slabels = slabels - np.min(slabels)
numclass = np.max(slabels)+1
print('number of class=',numclass)